In [2]:
import numpy as np
import pandas as pd
import math
import random
import subprocess
import copy
import multiprocessing

In [3]:
n, r, n2, size = [4,6, 16, 2**16]
print('n =', n)
print('r =', r)
print('n2 =', n2)
print('size =', size)

s_box=[3, 8, 11, 5, 6, 4, 14, 10, 2, 12, 1, 7, 9, 15, 13, 0]

n = 4
r = 6
n2 = 16
size = 65536


In [4]:
def xor_key(x, key):
    return x ^ key

In [5]:
def s_block(bits, s_box, x):
    if bits == n:
        return s_box[x]
    elif bits == n2:
        s=0
        for i in range(n):
            s += (s_box[x&15]<<4*i)
            x = x>>4
        return s

def reverse_s_block(bits, s_box, x):
    if bits == n:
        return s_box.index(x)
    elif bits == n2:
        s=0
        for i in range(n):
            s += (s_box.index(x&15)<<4*i)
            x = x>>4
        return s

In [6]:
def shifting(x):
    shifted_x = sum((1 << (4 * j + i)) for i in range(n) for j in range(n) if x & (1 << (4 * i + j)))
    return shifted_x

In [7]:
def Heys(x, key, printing=False):
    y = x
    
    for i in range(r):
        y = xor_key(y, key[i])
        y = s_block(n2, s_box, y)
        y = shifting(y)
        
        if printing:
            print(f"------- Round {i+1} -------")
            print(f"key = {key[i]}")
            print(f"y = {y}")
    
    y = xor_key(y, key[r])
    
    if printing:
        print(f"------- Round {7} -------")
        print(f"key = {key[r]}")
        print(f"y = {y}")
    
    return y

In [8]:
key = [random.randint(0, size) for _ in range(r + 1)]
print(key)

[32738, 5989, 184, 48262, 64712, 21596, 33634]


In [9]:
x = 1
Heys(x, key, True)

------- Round 1 -------
key = 32738
y = 38160
------- Round 2 -------
key = 5989
y = 47565
------- Round 3 -------
key = 184
y = 31049
------- Round 4 -------
key = 48262
y = 20398
------- Round 5 -------
key = 64712
y = 31867
------- Round 6 -------
key = 21596
y = 21742
------- Round 7 -------
key = 33634
y = 55180


55180

## Таблиця лінійних потенціалів

In [10]:
def mul_x_alpha(x, alpha):
    res = 0
    i = 0
    while i < n2:
        temp1 = ((x >> i) & 1)
        temp2 = ((alpha >> i) & 1)
        res ^= temp1 * temp2
        i += 1
    return res

In [11]:
def tableLP():
    LP = [[0 for i in range(n2)] for i in range(n2)]
    LP[0][0] = 1
    alpha = 1
    while alpha < n2:
        beta = 1
        while beta < n2:
            x = 0
            while x < n2:
                sx = s_block(n2, s_box, x)
                a_x = mul_x_alpha(x, alpha)
                b_sx = mul_x_alpha(sx, beta)
                LP[alpha][beta] += (-1) ** (a_x ^ b_sx)
                x += 1
            
            LP[alpha][beta] = (LP[alpha][beta]/n2)**2
            beta += 1
        alpha += 1
    return LP


In [12]:
LP=tableLP()

pd.DataFrame(LP, columns = ["B0", "B1", "B2", "B3", "B4", "B5", "B6", "B7", "B8", "B9", "B10", "B11", "B12", "B13", "B14", "B15"], index = ["A0", "A1", "A2", "A3", "A4", "A5", "A6", "A7", "A8", "A9", "A10", "A11", "A12", "A13", "A14", "A15"])

,B0,B1,B2,B3,B4,B5,B6,B7,B8,B9,B10,B11,B12,B13,B14,B15
A0,1,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000
A1,0,0.0625,0.0625,0.2500,0.0000,0.0625,0.0625,0.0000,0.0625,0.0000,0.0000,0.0625,0.0625,0.0000,0.2500,0.0625
A2,0,0.0625,0.0625,0.2500,0.0000,0.0625,0.0625,0.2500,0.0000,0.0625,0.0625,0.0000,0.0000,0.0625,0.0625,0.0000
A3,0,0.0000,0.0000,0.0000,0.2500,0.0000,0.2500,0.0000,0.0625,0.0625,0.0625,0.0625,0.0625,0.0625,0.0625,0.0625
A4,0,0.0000,0.0625,0.0625,0.0000,0.0000,0.0625,0.0625,0.0000,0.2500,0.0625,0.0625,0.2500,0.0000,0.0625,0.0625
A5,0,0.0625,0.2500,0.0625,0.0000,0.0625,0.0000,0.0625,0.0625,0.0000,0.0625,0.0000,0.0625,0.2500,0.0625,0.0000
A6,0,0.0625,0.0000,0.0625,0.2500,0.0625,0.0000,0.0625,0.0000,0.0625,0.2500,0.0625,0.0000,0.0625,0.0000,0.0625
A7,0,0.2500,0.0625,0.0625,0.0000,0.0000,0.0625,0.0625,0.0625,0.0625,0.2500,0.0000,0.0625,0.0625,0.0000,0.0000
A8,0,0.0000,0.0625,0.0625,0.0625,0.0625,0.0000,0.2500,0.0000,0.0000,0.0625,0.0625,0.0625,0.0625,0.0000,0.2500
A9,0,0.0625,0.0000,0.0625,0.0625,0.2500,0.0625,0.0000,0.0625,0.0000,0.0625,0.0000,0.2500,0.0625,0.0000,0.0625


In [13]:
def create_list(alpha):
    a = [0 for i in range(n)]
    Bq = []
    i = 0
    while i < n:
        a[n - i - 1] = int(alpha % n2)
        alpha /= n2
        i += 1
    b1 = []
    j1 = 0
    while j1 < n2:
        if LP[a[0]][j1] != 0:
            b1.append(j1)
        j1 += 1
    b2 = []
    j2 = 0
    while j2 < n2:
        if LP[a[1]][j2] != 0:
            b2.append(j2)
        j2 += 1
    b3 = []
    j3 = 0
    while j3 < n2:
        if LP[a[2]][j3] != 0:
            b3.append(j3)
        j3 += 1
    b4 = []
    j4 = 0
    while j4 < n2:
        if LP[a[3]][j4] != 0:
            b4.append(j4)
        j4 += 1
    j1 = 0
    while j1 < len(b1):
        j2 = 0
        while j2 < len(b2):
            j3 = 0
            while j3 < len(b3):
                j4 = 0
                while j4 < len(b4):
                    b = b4[j4] + (b3[j3] << 4) + (b2[j2] << 8) + (b1[j1] << 12)
                    q = LP[a[0]][b1[j1]] * LP[a[1]][b2[j2]] * LP[a[2]][b3[j3]] * LP[a[3]][b4[j4]]
                    Bq.append([b, q])
                    j4 += 1
                j3 += 1
            j2 += 1
        j1 += 1
    return Bq

In [14]:
p=0.00009

def PotentialSearch(alpha):
    G0 = dict()
    G0[alpha] = 1
    beta_dict = dict()
    t = 1

    while t < r:
        G1 = dict()
        for g in G0:
            p_i = G0[g]
            if g not in beta_dict:
                delta_beta = create_list(g)
                beta_dict[g] = delta_beta
            else:
                delta_beta = beta_dict[g]
                
            j = 0
            j_length = len(delta_beta)

            while j < j_length:
                delta = delta_beta[j]
                if delta[0] in G1:
                    G1[delta[0]] += p_i * delta[1]
                else:
                    G1[delta[0]] = p_i * delta[1]
                j += 1
        
        G0 = {shifting(g1): G1[g1] for g1 in G1 if G1[g1] > p}
        t += 1

    return G0

In [15]:
%%time
potential_search_result = {}
i = 1

while i <= n2:
    j = 0
    while j < 4:
        G = PotentialSearch(i << 4*j)  # alpha = i << 4*j
        if G:
            potential_search_result[i << 4*j] = G
            print(i << 4*j, G)
        j += 1
    i += 1

1 {68: 0.00011884569539688528, 1088: 9.566904918756336e-05, 17408: 0.00013692504944629036, 1092: 0.000120275974040851, 16388: 0.00014075170838623308}
16 {1092: 9.560445323586464e-05, 16388: 0.00012356904335319996, 17408: 0.00011494266800582409}
256 {32776: 0.00013572429088526405, 34816: 0.00012885159594588913, 2184: 9.82913188636303e-05}
2 {68: 0.0001484300009906292, 1088: 0.00010289315832778811, 17408: 0.00012649472046177834, 17472: 9.742628026288003e-05, 17476: 9.154358122032136e-05, 1092: 0.00013100681826472282, 16388: 0.00012891569349449128}
32 {68: 0.00012158788740634918, 1088: 9.30149108171463e-05, 17408: 0.00011161761358380318, 1092: 0.00010972656309604645, 16388: 0.00011500949040055275}
512 {2176: 9.142456110566854e-05, 32776: 0.00012220963253639638, 34816: 0.00012645646347664297, 136: 0.00011615455150604248, 2184: 0.00010654330253601074}
3 {68: 0.00013051251880824566, 1028: 0.0001381962065352127, 1088: 9.081755706574768e-05, 1092: 0.00015249136777129024, 16388: 0.0001089750585

2560 {2048: 0.0001351038517896086, 2176: 0.00017849394430413668, 32776: 0.00012137636372244742, 32896: 0.00011456492302386323, 32904: 9.22957515285816e-05, 34816: 0.00015526642096119758, 34944: 9.981756375054829e-05, 34952: 9.441309885005467e-05, 136: 0.0001318629431352747, 2184: 0.00012199480920571659}
40960 {34: 0.0001541774472570978, 544: 0.00012200514174764976, 8736: 9.248947026208043e-05, 51: 9.609890406636623e-05, 546: 0.00011507680028444156, 8194: 9.785274596652016e-05}
11 {17: 0.00011647236533463001, 34: 0.00017561507411301136, 544: 9.63675556704402e-05, 8736: 0.00010614041821099818, 51: 0.00013075473458457054, 273: 9.94254369288683e-05, 546: 0.00015004468150436878, 819: 0.00010642467356092311, 64: 0.00010136884156963788, 68: 0.0001838895695982501, 1088: 0.00020301695622038096, 16384: 0.00011139608977828175, 16452: 0.00014176957483869046, 17408: 0.00020300832329667173, 17472: 0.00014216173440217972, 17476: 0.00012507631254266016, 1092: 0.0001627882847969886, 16388: 0.0001679316

In [16]:
import pprint
pprint.pprint(potential_search_result)

{1: {68: 0.00011884569539688528,
     1088: 9.566904918756336e-05,
     1092: 0.000120275974040851,
     16388: 0.00014075170838623308,
     17408: 0.00013692504944629036},
 2: {68: 0.0001484300009906292,
     1088: 0.00010289315832778811,
     1092: 0.00013100681826472282,
     16388: 0.00012891569349449128,
     17408: 0.00012649472046177834,
     17472: 9.742628026288003e-05,
     17476: 9.154358122032136e-05},
 3: {68: 0.00013051251880824566,
     1028: 0.0001381962065352127,
     1088: 9.081755706574768e-05,
     1092: 0.00015249136777129024,
     16384: 0.00014248296793084592,
     16388: 0.0001089750585379079,
     16452: 0.000128988191136159,
     17408: 0.00021186668891459703,
     17476: 0.00010394075070507824},
 4: {68: 0.00010382091568317264,
     1092: 0.00015530359814874828,
     17408: 0.00010383184417150915,
     17476: 0.0001304151810472831},
 5: {17: 9.614601731300354e-05,
     34: 0.0001447834074497223,
     51: 0.00011949697505997392,
     68: 0.00018134897652544169

In [17]:
potential_res = [((alpha, beta), prob) for alpha in potential_search_result for beta, prob in potential_search_result[alpha].items()]
sorted_potential_result = sorted(potential_res, key=lambda x: x[1], reverse=True)[1:]

pprint.pprint(sorted_potential_result)
print('len(sorted_potential_result) = ', len(sorted_potential_result))

[((7, 1092), 0.0002937568351626396),
 ((6, 1092), 0.00027651370328385383),
 ((9, 1092), 0.0002764723903965205),
 ((1792, 2184), 0.00026229582726955414),
 ((2304, 2184), 0.0002576910483185202),
 ((1536, 2184), 0.00025122899387497455),
 ((7, 68), 0.0002479059621691704),
 ((144, 1092), 0.00023998599499464035),
 ((6, 68), 0.00023966740991454571),
 ((112, 1092), 0.00023865699768066406),
 ((96, 1092), 0.00023492681793868542),
 ((3, 17408), 0.00021186668891459703),
 ((768, 34816), 0.0002084351726807654),
 ((11, 1088), 0.00020301695622038096),
 ((11, 17408), 0.00020300832329667173),
 ((96, 68), 0.0002021655673161149),
 ((1792, 136), 0.00020202063024044037),
 ((10, 1088), 0.0001989736628047467),
 ((10, 34), 0.00019792653620243073),
 ((48, 17408), 0.00019313558004796505),
 ((9, 68), 0.00019061930652242154),
 ((13, 68), 0.00019020464969798923),
 ((112, 68), 0.00018987059593200684),
 ((7, 17476), 0.0001894177657959517),
 ((2816, 34816), 0.0001876628157333471),
 ((1536, 136), 0.00018572641420178115

## Пошук ключів

In [15]:
print(key)

messages = []

i = 0
while i < 10000:
    random_int = random.randint(1, size - 1)
    messages.append(random_int)
    i += 1

l = 100

[32738, 5989, 184, 48262, 64712, 21596, 33634]


In [ ]:
%%time

num_processes = multiprocessing.cpu_count()
print(num_processes)

def process_messages(k):
    result = np.array([shift.result(s.result(pk.result(x,k))) for x in messages], dtype=np.uint16)
    return result


one_round_messages=[]
with  multiprocessing.Pool(processes=num_processes) as pool:
    
    one_round_messages = pool.map(process_messages, range(size))
    pool.close()
    pool.join()

In [8]:
one_round_messages

[]

In [ ]:
def bit_count(n):
    count = 0
    while n:
        count += n & 1
        n >>= 1
    return count


all_alpha = set()
for a, p in sorted_potential_result:
    all_alpha.add(a[0])

alpha_multipl = {}
for i in all_alpha:
    alpha_multipl[i] = np.array([bit_count(i & j) & 1 for j in range(size)], dtype=bool)

all_beta = set()
for b, p in sorted_potential_result:
    all_beta.add(b[1])

beta_multipl = {}
for i in all_beta:
    beta_multipl[i] = np.array([bit_count(i & j) & 1 for j in range(size)], dtype=bool)


In [ ]:
def count_u(args):
    one_round_messages, cipher_messages, alpha, beta = args
    u = sum((-1 if (alpha_multipl[alpha][x] ^ beta_multipl[beta][y]) == 1 else 1) for x, y in zip(one_round_messages, cipher_messages))
    return abs(u)

In [ ]:
def attack(cm):
    candidates = {}
    num_processes = multiprocessing.cpu_count()
    for (alpha, beta), p in sorted_potential_result:
        alpha_beta_keys=[]
        with multiprocessing.Pool(processes=num_processes) as pool:
            arg=[(one_round_messages[i], cipher_messages, alpha, beta) for i in range(size)]
            alpha_beta_keys = pool.map(count_u,arg)
            pool.close()
            pool.join()
            c=l
            temp = []
        
            for u in range(max(alpha_beta_keys), 0, -1):
                keys_same_u = [i for i in range(len(alpha_beta_keys)) if alpha_beta_keys[i]==u]
                if len(keys_same_u)<=c:
                    temp.extend(keys_same_u)
                    c -= len(keys_same_u)
                else:
                    t = len(keys_same_u)
                    i = 0
                    while i < t - c:
                        r = random.randint(0, len(keys_same_u)-1)
                        temp.append(keys_same_u[r])
                        keys_same_u.remove(r)
                        i += 1
                    break
            candidates[(alpha,beta)]=np.array(temp, dtype = np.uint16)
    
    keys = {}
    for (alpha, beta), p in sorted_potential_result:
        for k in candidates[(alpha, beta)]:
            keys[k] = keys.get(k, 0) + 1

    keys = sorted(keys.items(), key = lambda x: x[1], reverse = True)
    return keys[:10]

## Атака на реалізований шифр Хейса

In [ ]:
%%time
cipher_messages=np.array([Heys(x,key, False) for x in messages], dtype = np.uint16)

In [ ]:
%%time
keys = attack(cipher_messages)

In [ ]:
print(keys)

## Атака на програму

In [ ]:
%%time
cipher_texts=[]
with open(f'./input.bin', 'wb') as f:
        for text in messages:
            f.write(text.to_bytes(2, 'little'))

subprocess.run(f'./heys.bin e 07 ./input.bin ./output.bin', shell = True)

with open(f'./output.bin', 'rb') as f:
        line = f.read()
        concatenated_cipher_text = int.from_bytes(line, byteorder='little')

        for i in range(len(line) // 2):
            cipher_texts.append((concatenated_cipher_text >> (i * 16)) & (2**16 - 1))

cipher_messages=np.array(cipher_texts, dtype = np.uint16)

In [ ]:
%%time
keys = attack(cipher_messages)

In [ ]:
print(keys)